In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
import os
import warnings
from skimage import io as skimio
from skimage import color as skimcolor
from datetime import datetime
from datetime import date
import pickle

from PIL import Image
import requests
from io import BytesIO

from segmentation.imageModifiers import *
from segmentation.plottingFuncs import *
from segmentation.projEdgeBreaks import *

mpl.rcParams["figure.figsize"] = (15, 15)

# Read in all classifications

In [ ]:
sv_fold = "../data/ASM/"
if not os.path.isdir(sv_fold+"Images"):
    os.mkdir(sv_fold+"Images")

full_sv = os.getcwd().replace("\\", "/")
full_sv = full_sv + "data/ASM/Images/"

if os.path.exists("../data/ASM/newclas.pkl"):
    data = pickle.load(open("../data/ASM/newclas.pkl", "rb"))
    print("\n{0} data entries loaded".format(len(data)))
else:
    print("File doesn't exist, run \"preprocess_ASM_csv.py\" first")

# Preprocess by splitting image into sections

In [ ]:
def readImg(url, grey=True):
    response = requests.get(loc)
    img = Image.open(BytesIO(response.content))
    img = img.convert("LA")
    return img

In [ ]:
img = readImg(loc)
# Image.open(loc)

plt.imshow(img)
plt.show()

In [ ]:
val = clas1["annotations_dict"][0]["value"]
val
for v in val:
    if v["frame"] == 0:
        pts = v["points"]
        print(pts)

In [ ]:
for k in subj1["locations_dict"].iloc[0].keys():
    val = clas1["annotations_dict"][0]["value"]
    pts = [v["points"] for v in val if v["frame"] == k]
    
    print(k)
# clas1.annotations_dict

In [ ]:
pts

# Create training csv w/ transcription

# Prediction current section

# Train on current section

# Delete all images to save space